In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from bnr_ml.nnet import layers
from bnr_ml.objectdetect import yolo
from bnr_ml.nnet.updates import momentum
import theano
from theano import tensor as T
from collections import OrderedDict
from tqdm import tqdm
import time

import lasagne
import lasagne.layers as layers

import pickle as pk

import pdb

In [3]:
def gen_num_objs(max_obj):
    return int(1 + max_obj * np.random.rand())

In [4]:
def gen_classes(num_obj, num_class):
    return np.random.randint(num_class, size=(num_obj,))

In [5]:
def gen_circle(shape):
#     pdb.set_trace()
    row, col = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    row, col = row.astype(np.float64), col.astype(np.float64)
    row = (row - row.shape[1] / 2) / (row.shape[1] / 2)
    col = (col - col.shape[0] / 2) / (col.shape[0] / 2)
    
    
    im = np.zeros(shape + (1,))
    for i in range(im.shape[2]):
        in_ellipse = (row**2 + col**2) < 1
#         im[:,:,i][in_ellipse] = (np.random.gamma(.7) + np.random.gamma(.4) * np.random.randn(*shape))[in_ellipse]
        im[:,:,i][in_ellipse] = np.ones(shape)[in_ellipse]
    im[im < 0] = 0.
    im[im >= 1] = 1.
    return im

In [6]:
def gen_rect(shape):
    im = np.zeros(shape + (3,))
    for i in range(3):
        im[:,:,i] = (np.random.gamma(.7) + np.random.gamma(.4) * np.random.randn(*shape))
    im[im < 0] = 0.
    im[im > 1] = 1.
    return im

In [7]:
def gen_triangle(shape):
    row, col = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    row, col = row.astype(np.float64), col.astype(np.float64)
    row = row / row.shape[1]
    col = col / col.shape[0]
    
    
    im = np.zeros(shape + (1,))
    if np.random.rand() < .5:
        in_triangle = row + col < 1
    else:
        in_triangle = row + col > 1
    for i in range(im.shape[2]):
        im[:,:,i][in_triangle] = np.ones(shape)[in_triangle]
#         im[:,:,i][in_triangle] = (np.random.gamma(.7) + np.random.gamma(.4) * np.random.randn(*shape))[in_triangle]
    im[im < 0] = 0.
    im[im >= 1] = 1.
    return im

In [8]:
def gen_pos_shape(min_shape, im_shape):
    shape = (
        int(min_shape[0] + (im_shape[0] - min_shape[0]) * np.random.rand()),
        int(min_shape[1] + (im_shape[1] - min_shape[1]) * np.random.rand())
    )
    pos = (
        int((im_shape[0] - shape[0]) * np.random.rand()),
        int((im_shape[1] - shape[1]) * np.random.rand())
    )
    return shape, pos

In [9]:
def gen_scene(max_obj, num_class, min_shape, im_shape):
    num_obj = gen_num_objs(max_obj)
    classes = gen_classes(num_obj, num_class)
    im = np.random.gamma(.4) * np.random.randn(*(im_shape + (1,)))
    im[im < 0] = 0.
    im[im > 1] = 1.
    y = np.zeros((4 + num_class,))
    
    for cls in classes:
        shape, pos = gen_pos_shape(min_shape, im_shape)
        if cls == 0:
            shp = gen_circle(shape)
        elif cls == 1:
            shp = gen_triangle(shape)
        elif cls == 2:
            shp = gen_triangle(shape)
        else:
            pass
        im[pos[0]:pos[0]+shape[0],pos[1]:pos[1]+shape[1],:] += shp
        y[:4] = pos + shape
        y = y.astype(np.float64)
        y[[0,2]] /= im_shape[0]
        y[[1,3]] /= im_shape[1]
        y[-num_class + cls] += 1.
    return im / np.max(im), y

In [10]:
def gen_dataset(size, shape, max_obj=1):
    X, y = np.zeros((size,1) + shape), np.zeros((size,4+2))
    for i in range(size):
        dat = gen_scene(max_obj, 2, (20, 20), shape)
        X[i], y[i] = dat[0].swapaxes(2,1).swapaxes(1,0), dat[1]
    return X.astype(theano.config.floatX), y.astype(theano.config.floatX)

In [11]:
X,y = gen_dataset(1000, (100,100))

In [ ]:
N = np.int_(X.shape[0] * np.random.rand())
plt.imshow(X[N,0], 'gray')
plt.show()

print y[N,:4]

In [12]:
input = T.tensor4('input')
# input = theano.shared(X[:10], name='dat', borrow=True)

In [13]:
l1 = layers.InputLayer((None,1,100,100), input_var=input)
l2 = layers.Conv2DLayer(
    l1,
    8,
    (3,3)
)
l3 = layers.Conv2DLayer(
    l2,
    8,
    (3,3)
)
l4 = layers.Pool2DLayer(
    l3,
    (3,3)
)
l5 = layers.Conv2DLayer(
    l4,
    16,
    (3,3)
)
l6 = layers.Conv2DLayer(
    l5,
    16,
    (3,3)
)
l7 = layers.Pool2DLayer(
    l6,
    (3,3)
)

le1 = layers.Conv2DLayer(
    l7,
    16,
    (3,3)
)
le2 = layers.Conv2DLayer(
    le1,
    16,
    (3,3)
)
le3 = layers.Pool2DLayer(
    le2,
    (2,2)
)

l8 = layers.DenseLayer(
    layers.dropout(le3, p=.2),
    100
)
l9 = layers.DenseLayer(
    layers.dropout(l8, p=.2),
    100,
)
l10 = layers.DenseLayer(
    layers.dropout(l9, p=.2),
    2,
    nonlinearity=lasagne.nonlinearities.softmax
)

In [14]:
with open('../weights/weights.pkl', 'rb') as f:
    weights = pk.load(f)
    weights = [w.astype(theano.config.floatX) for w in weights]

In [15]:
layers.set_all_param_values(l10, weights)

In [16]:
olayer = layers.DenseLayer(
    layers.dropout(l8, p=.2),
    2 * 2 * (10 + 2)
)

In [ ]:
# with open('../weights/yolo_weights.pkl', 'wb') as f:
#     pk.dump(layers.get_all_param_values(olayer), f)
with open('../weights/yolo_weights.pkl', 'rb') as f:
    weights = pk.load(f)
    weights = [w.astype(theano.config.floatX) for w in weights]
layers.set_all_param_values(olayer, weights)

In [17]:
net = {}
net['input'] = l1
net['l2'] = l2
net['l3'] = l3
net['l4'] = l4
net['l5'] = l5
net['l6'] = l6
net['l7'] = l7
net['le1'] = le1
net['le2'] = le2
net['le3'] = le3
net['l8'] = l8
net['output'] = olayer

In [61]:
reload(layers)
reload(yolo)

<module 'bnr_ml.objectdetect.yolo' from '/usr/local/python/bnr_ml/objectdetect/yolo.py'>

In [62]:
yl = yolo.YoloObjectDetector(net, (None,1,100,100), 2, (2,2), 2)

In [63]:
output_fn = theano.function([yl.input], yl.output)

In [70]:
# target = theano.shared(y[:10], name='gt', borrow=True)
# output = T.as_tensor(output_fn(X[:1000]))
# target = T.as_tensor(y[:1000])

output = T.tensor4('output')
target = T.matrix('target')

In [71]:
cost = yl._get_cost_optim(output, target, yl.S, yl.B, yl.num_classes)
# cost_fn = theano.function([yl.input, target], T.grad(cost, yl.output))

> /usr/local/python/bnr_ml/objectdetect/yolo.py(130)_get_cost_optim()
-> T.sum((pred[:,h_idx,:,:][is_max.nonzero()].reshape((truth.shape[0],-1)).sqrt() - truth[:,[3]].sqrt())**2) + \
(Pdb) c


In [72]:
fn = theano.function([output, target], T.grad(cost, output))

In [74]:
y[:100].shape

(100, 6)

In [73]:
fn(output_fn(X[:100]), y[:100]).shape

ValueError: CudaNdarray_CopyFromCudaNdarray: need same dimensions for dim 0, destination=100, source=1
Apply node that caused the error: GpuAdvancedIncSubtensor1{inplace,set}(GpuDimShuffle{1,0,2,3}.0, CudaNdarrayConstant{[[[[ 0.]]]]}, TensorConstant{[1 6]})
Toposort index: 220
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, (True, True, True, True)), TensorType(int64, vector)]
Inputs shapes: [(12, 100, 2, 2), (1, 1, 1, 1), (2,)]
Inputs strides: [(400, 4, 2, 1), (0, 0, 0, 0), (8,)]
Inputs values: ['not shown', CudaNdarray([[[[ 0.]]]]), array([1, 6])]
Outputs clients: [[GpuDimShuffle{1,0,2,3}(GpuAdvancedIncSubtensor1{inplace,set}.0)]]

Debugprint of the apply node: 
GpuAdvancedIncSubtensor1{inplace,set} [id A] <CudaNdarrayType(float32, 4D)> ''   
 |GpuDimShuffle{1,0,2,3} [id B] <CudaNdarrayType(float32, 4D)> ''   
 | |GpuIncSubtensor{InplaceInc;::, int64::, ::, ::} [id C] <CudaNdarrayType(float32, 4D)> ''   
 |   |GpuElemwise{Add}[(0, 0)] [id D] <CudaNdarrayType(flo

In [51]:
cost_fn(X[:10], y[:10])

array([[[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 7.52113914,  5.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  6.09794855],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [35]:
train_loss, test_loss = yl.train(
    X[:5000],
    y[:5000],
    lr=1e-3,
    momentum=0.5,
    train_test_split=0.9,
    epochs=10,
    batch_size=4,
    lmbda_coord=5.,
    lmbda_noobj=.5,
    target=None
)

Getting cost...
Creating cost variable took 0.4074 seconds
Compiling...


  0%|          | 0/10 [00:00<?, ?it/s]

Compiling functions took 21.0711 seconds
Beginning training...


ValueError: CudaNdarray_CopyFromCudaNdarray: need same dimensions for dim 0, destination=4, source=1
Apply node that caused the error: GpuAdvancedIncSubtensor1{inplace,set}(GpuAdvancedIncSubtensor1{inplace,inc}.0, CudaNdarrayConstant{[[[[ 0.]]]]}, TensorConstant{[1 6]})
Toposort index: 253
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, (True, True, True, True)), TensorType(int64, vector)]
Inputs shapes: [(12, 4, 2, 2), (1, 1, 1, 1), (2,)]
Inputs strides: [(16, 4, 2, 1), (0, 0, 0, 0), (8,)]
Inputs values: ['not shown', CudaNdarray([[[[ 0.]]]]), array([1, 6])]
Outputs clients: [[GpuDimShuffle{1,0,2,3}(GpuAdvancedIncSubtensor1{inplace,set}.0)]]

Debugprint of the apply node: 
GpuAdvancedIncSubtensor1{inplace,set} [id A] <CudaNdarrayType(float32, 4D)> ''   
 |GpuAdvancedIncSubtensor1{inplace,inc} [id B] <CudaNdarrayType(float32, 4D)> ''   
 | |GpuAdvancedIncSubtensor1{inplace,inc} [id C] <CudaNdarrayType(float32, 4D)> ''   
 | | |GpuAdvancedIncSubtensor1{inplace,inc}

Changed the regularization parameter to improve the classification accuracy.

In [ ]:
testl, trainl = test_loss, train_loss
# testl, trainl = np.concatenate((testl, test_loss)), np.concatenate((trainl, train_loss))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(trainl[10:], 'r', linewidth=2.0)
plt.plot(testl[10:], 'b', linewidth=2.0)
plt.show()

In [ ]:
pred_fn = theano.function([yl.input], yl.output)

In [ ]:
Xtest, ytest = gen_dataset(1000, (100,100), max_obj=1)

In [ ]:
ti = time.time()
tmp = pred_fn(Xtest[:500])
tf = time.time()
print ((tf - ti) / tmp.shape[0])

In [ ]:
N = np.int_(tmp.shape[0] * np.random.rand())
best, confs = find_best_coords_2(tmp[N], (100,100))
im = Xtest[N,0]
for i in range(best.shape[0]):
    im = draw_coords(im, best[i])
plt.imshow(im)
print find_best_coords(tmp[N], (100,100))
print y[N,:4]
for i in range(confs.size):
    print confs[i]
# prb = np.max(probs)
# idx = np.argmax(probs)
# if idx == 0:
#     print('Circle with confidence: %.3f' % prb)
# else:
#     print('Triangle with confidence: %.3f' % prb)
# print iou

In [ ]:
def find_best_coords(output, shape):
    idx = np.argmax(output[[4,9],:,:])
    coord = np.unravel_index(idx, (2,2,2))
    best = np.copy(output[coord[0] * 5:coord[0]*5 + 4, coord[1], coord[2]])
    best[0] += float(coord[1]) / 2
    best[1] += float(coord[2]) / 2
    return best, output[-2:,coord[1],coord[2]], output[[4,9],:,:][coord[0],coord[1],coord[2]]

In [ ]:
def find_best_coords_2(output, shape):
    best = np.zeros((4,4))
    confs = np.zeros((4,))
    cnt = 0
    for i in range(2):
        for j in range(2):
            pred = output[:,i,j]
            idx = np.argmax(pred[[4,9]])
            coord = pred[idx * 5:idx*5 + 4]
            conf = np.max(pred[-2:]) #* pred[idx*5 + 4]
#             conf = pred[idx*5 + 4]
            best[cnt] = coord
            confs[cnt] = conf
            cnt += 1
    return best, confs

In [ ]:
find_best_coords_2(tmp[N], (100,100))

In [ ]:
def draw_coords(im, coord):
    coord[[0,2]] *= im.shape[0]
    coord[[1,3]] *= im.shape[1]
    coord = coord.astype(np.int32)
    if im.shape.__len__() < 3:
        im = np.copy(im)
        im = im.reshape(im.shape + (1,))
        im = np.concatenate((im,im,im), axis=2)
    def bound(num, lb, ub):
        return min(ub, max(num,lb))
    ri,rf = bound(coord[0], 0, im.shape[0]-1), bound(coord[0]+coord[2], 0, im.shape[0]-1)
    ci,cf = bound(coord[1], 0, im.shape[1]-1), bound(coord[1]+coord[3], 0, im.shape[1]-1)
    im[ri:rf, ci,0] = 1.0
    im[ri:rf, cf,0] = 1.0
    im[ri, ci:cf,0] = 1.0
    im[rf, ci:cf,0] = 1.0
    return im

In [ ]:
a = draw_coords(X[N,0], find_best_coords(tmp[N], (100,100))[0])

In [ ]:
a.shape

In [ ]:
plt.imshow(a)

In [ ]:
print find_best_coords(tmp[N], (100,100))

In [ ]:
np.argmax(tmp[0,[4,9],:,:])

In [ ]:
np.unravel_index(4, (2,2,2))

In [ ]:
tmp[0,[4,9],:,:][1,0,0]

In [ ]:
act_fn = theano.function([input], layers.get_output(le3))

In [ ]:
tmp = act_fn(X[:20])

In [ ]:
tmp.max()

In [ ]:
Xtest, ytest = gen_dataset(100, (100,100))

In [ ]:
preds = pred_fn(Xtest)

In [ ]:
N = np.int_(Xtest.shape[0] * np.random.rand())
fig = plt.figure(figsize=(7,7))
plt.imshow(Xtest[N,0], 'gray')
plt.show()

for i in range(2):
    for j in range(2):
        print preds[N,:,i,j]

In [ ]:
ytest[N]